# Diabetic Retinopathy Classification Using MobileNetV2

# ABSTRACT

Deep learning has been proposed as one of the automated solutions for diabetic retinopathy (DR) severity classification problem. However, most of the successful deep learning models are based on large convolutional neural network (CNN) architectures, requiring a vast volume of training data as well as dedicated computational resources. In this study, we used MobileNetV2 architecture, which was considered a small-scale architecture (4.2 million trainable parameters), to perform DR classification task in APTOS 2019 dataset (3662 color retinal images). We used the generic MobileNetV2 pre-trained weights from ImageNet as initialization and implemented data augmentation.

# Import Library

In [ ]:
# Import Library

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from PIL import Image

from keras import layers
from tensorflow.keras import applications 
from keras.applications import MobileNetV2
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam


from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score, confusion_matrix

from tqdm import tqdm

# Loading Data and Exploration

## Loading data

Loading the csv data which contains the images file name and its labels. In this project I am using data that already splitted for training and validation.

In [ ]:
train_df = pd.read_csv('../input/valid-and-test-ta/x_train_8.csv')
valid_df = pd.read_csv('../input/valid-and-test-ta/x_valid_8.csv')
print(train_df.shape)
print(valid_df.shape)
train_df.head()

In [ ]:
train_df['diagnosis'].value_counts()
train_df['diagnosis'].hist()

## Displaying some Sample Images

Display the color retinal images with its label as the title

In [ ]:
def display_samples(df, columns=4, rows=3):
    fig=plt.figure(figsize=(5*columns, 4*rows))

    for i in range(columns*rows):
        image_path = df.loc[i,'id_code']
        image_id = df.loc[i,'diagnosis']
        img = cv2.imread(f'../input/aptos2019-blindness-detection/train_images/{image_path}.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        fig.add_subplot(rows, columns, i+1)
        plt.title(image_id)
        plt.imshow(img)
    
    plt.tight_layout()

display_samples(train_df)

# Pre-processing

## Sampling Data

Data is sampled into 700 data for each class

In [ ]:
#resample
from sklearn.utils import resample
X=train_df
normal=X[X.diagnosis==0]
mild=X[X.diagnosis==1]
moderate=X[X.diagnosis==2]
severe=X[X.diagnosis==3]
pdr=X[X.diagnosis==4]

#downsampled
mild = resample(mild,
                replace=True, # sample with replacement
                n_samples=700, # match number in majority class
                random_state=2020) # reproducible results
moderate = resample(moderate,
                    replace=False, # sample with replacement
                    n_samples=700, # match number in majority class
                    random_state=2020) # reproducible results
severe = resample(severe,
                  replace=True, # sample with replacement
                  n_samples=700, # match number in majority class
                  random_state=2020) # reproducible results
normal = resample(normal,
                  replace=False, # sample with replacement
                  n_samples=700, # match number in majority class
                  random_state=2020) # reproducible results
pdr = resample(pdr,
               replace=True, # sample with replacement
               n_samples=700, # match number in majority class
               random_state=2020) # reproducible results    

# combine minority and downsampled majority
sampled = pd.concat([normal, mild, moderate, severe, pdr])

train_df = sampled
train_df = train_df.sample(frac=1).reset_index(drop=True)

## Resize Images 
We will resize the images to 224x224 pixel (MobileNetV2 default input resolution), then create a single numpy array to hold the data (because the data is not too much).

In [ ]:
def preprocess_image(image_path, desired_size=224):
    im = Image.open(image_path)
    im = im.resize((desired_size, )*2, resample=Image.BILINEAR)
    
    return im

N = train_df.shape[0]
x_train = np.empty((N, 224, 224, 3), dtype=np.float32)

for i, image_id in enumerate(tqdm(train_df['id_code'])):
    x_train[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/train_images/{image_id}.png'
    )
    
N = valid_df.shape[0]
x_val = np.empty((N, 224, 224, 3), dtype=np.float32)

for i, image_id in enumerate(tqdm(valid_df['id_code'])):
    x_val[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/train_images/{image_id}.png'
    )

# Image Generator

Using Keras ImageDataGenerator to generate the images (x_train/valid) and its labels (y_train/valid)

In [ ]:
y_train = train_df['diagnosis']
y_val = valid_df['diagnosis']
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

In [ ]:
BATCH_SIZE = 32

def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.1,  # set range for random zoom
        rotation_range = 360,
        fill_mode='constant',
        cval=0.,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
    )

# Using generator
data_generator = create_datagen().flow(x_train, y_train, batch_size=BATCH_SIZE, seed=2019)

# Quadratic Weighted Kappa

Quadratic Weighted Kappa (QWK, the greek letter $\kappa$), also known as Cohen's Kappa, is the official evaluation metric. For our kernel, we will use a custom callback to monitor the score, and plot it at the end.

## What is Cohen Kappa?

According to the [wikipedia article](https://en.wikipedia.org/wiki/Cohen%27s_kappa), we have
> The definition of $\kappa$ is:
> $$\kappa \equiv \frac{p_o - p_e}{1 - p_e}$$
> where $p_o$ is the relative observed agreement among raters (identical to accuracy), and $p_e$ is the hypothetical probability of chance agreement, using the observed data to calculate the probabilities of each observer randomly seeing each category.

This metric is used because if we just using accuracy as the metric, it will give spurious results (because the data is imbalance). The QWK is more fit to the problem.

## Creating keras callback for QWK

Because our main metric in this problem is QWK, we have to make a function that calculate the QWK for every epoch and also saving the model that got the highest score. 

In [ ]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        X_val, y_val = self.validation_data[:2]
        
        y_pred = self.model.predict(X_val)
        y_pred = np.clip(y_pred,0,4)
        y_pred = y_pred.astype(int)

        _val_kappa = cohen_kappa_score(
            y_val,
            y_pred, 
            weights='quadratic'
        )

        self.val_kappas.append(_val_kappa)

        print(f"val_kappa: {_val_kappa:.4f}")
        
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save('model.h5')

        return
    
kappa_metrics = Metrics()

# Model: MobileNetV2

I am using MobileNetV2 architecture in this project. The architecture is adopted from [1], where they choose 1.3 as the width multiplier/ alpha (MobileNetV2 hyperparameter). They customize the last layer of the model to become 2 dense layer with 256 nodes and 1 nodes output layer. They use regression approach for this problem because diabetic retinopathy severity is an ordinal variables. Linear activation function is used in the output layer.

In [ ]:
mobilenet = MobileNetV2(
    alpha = 1.3,
    weights='../input/valid-and-test-ta/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.3_224_no_top.h5',
    include_top=False,
    input_shape=(224,224,3)
)

In [ ]:
def build_model():
    model = Sequential()
    model.add(mobilenet)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256))
    model.add(layers.Dense(256))
    model.add(layers.Dense(1))
    
    model.compile(
        loss='mse',
        optimizer=Adam(lr=0.0001),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
model = build_model()
model.summary()

# Training & Evaluation

## Training

I trained the model with 100 epoch, mean squared error (mse) as the loss function and Adam as the optimizer

In [ ]:
history = model.fit_generator(
    data_generator,
    steps_per_epoch=x_train.shape[0] / BATCH_SIZE,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=[kappa_metrics]
)

In [ ]:
#plotting accuracies and loss
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['acc', 'val_acc']].plot()
history_df.to_csv('history.csv', index = False)

In [ ]:
#plotting the QWK score
plt.plot(kappa_metrics.val_kappas)

## Evaluation

After training phase and already have the model with highest QWK score, we have to evaluate the model performance using confusion matrix to see whether the model is working properly or not

In [ ]:
model.load_weights('model.h5')
y_val_pred = model.predict(x_val)
#clipping the value to range of 0-4, and round it to the nearest integer
y_val_pred = np.clip(y_val_pred,0,4).astype(int)

In [ ]:
labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
cnf_matrix = confusion_matrix(valid_df['diagnosis'].astype('int'), y_val_pred)
df_cm = pd.DataFrame(cnf_matrix, index=labels, columns=labels)
plt.figure(figsize=(16, 7))
sns.heatmap(df_cm, annot=True, cmap="Blues")
plt.show()

In [ ]:
kappa_val = cohen_kappa_score(
            valid_df['diagnosis'].astype('int'),
            y_val_pred, 
            weights='quadratic'
        )
print(kappa_val)

References:

[1] J. Gao, C. Leung and C. Miao, "Diabetic Retinopathy Classification Using an Efficient Convolutional Neural Network," in 2019 IEEE International Conference on Agents (ICA), Jinan, China, 2019, pp. 80-85.